### Dataset descargado desde
## https://ieee-dataport.org/documents/smart-defender-dataset

La descripción de los campos que componen los datos del dataset están disponible en esa URL

# PARTE 1: Uso de Spark para análisis de datos, Dataframes #

#### Importamos las librerías que vamos a necesitar.

In [32]:
import os
from pyspark.sql.functions import lit, col
from pyspark.sql.functions import udf

In [2]:
# Mostrar la versión de Spark usada
print("Spark Version: " + spark.version)
# Datos de la sesión spark
spark

Spark Version: 2.4.4


In [3]:
# Cargar el dataset en un Dataframe
data_path = os.path.join("data","dataset_descriptor.csv.gz")
cyber_df =  spark.read.option("header","true").csv(data_path)

In [4]:
# Mostrar el schema heredado
cyber_df.printSchema()

root
 |-- ip_proto: string (nullable = true)
 |-- ip_len_mean: string (nullable = true)
 |-- ip_len_median: string (nullable = true)
 |-- ip_len_var: string (nullable = true)
 |-- ip_len_std: string (nullable = true)
 |-- ip_len_entropy: string (nullable = true)
 |-- ip_len_cv: string (nullable = true)
 |-- ip_len_cvq: string (nullable = true)
 |-- ip_len_rte: string (nullable = true)
 |-- ip_ttl_mean: string (nullable = true)
 |-- ip_ttl_median: string (nullable = true)
 |-- ip_ttl_var: string (nullable = true)
 |-- ip_ttl_std: string (nullable = true)
 |-- ip_ttl_entropy: string (nullable = true)
 |-- ip_ttl_cv: string (nullable = true)
 |-- ip_ttl_cvq: string (nullable = true)
 |-- ip_ttl_rte: string (nullable = true)
 |-- sport_mean: string (nullable = true)
 |-- sport_median: string (nullable = true)
 |-- sport_var: string (nullable = true)
 |-- sport_std: string (nullable = true)
 |-- sport_entropy: string (nullable = true)
 |-- sport_cv: string (nullable = true)
 |-- sport_cvq: 

In [5]:
# Mostrar el número de registros del dataset
total_count = cyber_df.count()
total_count

45500

In [6]:
# Q1. ¿Cuantas clases de tipo de tráfico hay clasificadas en el campo label1?
label1_values = cyber_df.select("Label1").distinct().collect()
for label in label1_values:
    print(label.Label1)

http_flood
http_slow_body
tcp_fin_flood
http_slow_range
normal
udp_flood
http_slow_read
tcp_syn_ack_flood
tcp_syn_flood
tcp_ack_flood
http_slow_headers


In [7]:
cyber_df.select("Label1").distinct().count()

11

In [8]:
# Para obtener el mismo resultado que en el PDF debemos usar show(11)
cyber_df.select("Label1").distinct().show(n=11, truncate=False)

+-----------------+
|Label1           |
+-----------------+
|http_flood       |
|http_slow_body   |
|tcp_fin_flood    |
|http_slow_range  |
|normal           |
|udp_flood        |
|http_slow_read   |
|tcp_syn_ack_flood|
|tcp_syn_flood    |
|tcp_ack_flood    |
|http_slow_headers|
+-----------------+



In [9]:
# Q2. ¿Cuantas clases de tipo de tráfico hay clasificadas en el campo label3?
label3_values = cyber_df.select("Label3").distinct().collect()
for label in label3_values:
    print(label.Label3)

http_flood
tcp_flood
normal
udp_flood
http_slow


In [10]:
cyber_df.select("Label3").distinct().count()

5

In [11]:
# Para obtener el mismo resultado que en el PDF podemos usar show(5)
cyber_df.select("Label3").distinct().show(n=5, truncate=False)

+----------+
|Label3    |
+----------+
|http_flood|
|tcp_flood |
|normal    |
|udp_flood |
|http_slow |
+----------+



In [12]:
#Q3 ¿Que porcentaje de tráfico está catalogado como anormal? Entiendase por anormal aquel que no está etiquetado como normal.
anormal_trafic_count = cyber_df.where(cyber_df.Label3 != lit("normal")).count()
anormal_trafic_percentage = (anormal_trafic_count / total_count) * 100
print("% 5.2f %%" % (anormal_trafic_percentage))

 49.26 %


In [31]:
# Q4 Mostrar los porcentajes de trafico sobre el total asociados a cada tipo de etiqueta de tráfico 
# (usar el campo genérico Label3 y no el detallado Label 1)
# Mostrar un diagrama con estos porcentajes (bar plot)
trafic_per_label3 = cyber_df.groupBy(cyber_df.Label3).count()
trafic_label3_percentage = trafic_per_label3.select(trafic_per_label3.Label3, ((col("count") / lit(total_count)) * lit(100)).alias("percentage"))
trafic_label3_percentage.show()

+----------+-------------------+
|    Label3|         percentage|
+----------+-------------------+
|http_flood| 0.7626373626373626|
| tcp_flood|  32.94065934065934|
|    normal|  50.74285714285715|
| udp_flood| 15.151648351648353|
| http_slow|0.40219780219780216|
+----------+-------------------+



In [27]:
def percent(a, b):
    (a / lit(b)) * lit(100)

In [ ]:
example_udf = udf(example, LongType())
df.withColumn('result', example_udf(df.address1, df.address2))

In [28]:
trafic_per_label3.select(trafic_per_label3.Label3, percent(col("count"), total_count))

TypeError: Invalid argument, not a string or column: None of type <class 'NoneType'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [16]:
trafic_per_label3.count

<bound method DataFrame.count of DataFrame[Label3: string, count: bigint]>

In [ ]:
# Q5 Identificar que tipo de tráfico de red está incluido en el dataset (usar el campo ip_proto y convertir 
# ese valor al real que debería tener, es decir, un entero en el rango definido por el IANA)
# https://www.iana.org/assignments/protocol-numbers/protocol-numbers.xhtml


In [ ]:
# Q6 Calcular la cantidad total (suponer que el dato a aculumar para cada paquete es ip_len_mean) 
# de bytes transmitidos agrupados por protocolo


In [ ]:
# Q7 Indicar cual es tráfico anómalo en UDP que usa más tráfico de red. Presentar los resultados en orden demayor a menor


In [ ]:
# Q8 Indicar cual es tráfico anómalo en TCP que usa más tráfico de red. Presentar los resultados en orden demayor a menor


# PARTE 2: Uso de Spark para análisis de datos, Spark SQL #

Contestar a las cuestiones anteriores empleando Spark SQL

In [ ]:
# Crear la vista en memoria y añadir una celda por cada cuestión que se debe responder

In [ ]:
# Q1. ¿Cuantas clases de tipo de tráfico hay clasificadas en el campo label1?

In [ ]:
# Q2. ¿Cuantas clases de tipo de tráfico hay clasificadas en el campo label3?

In [ ]:
#Q3 ¿Que porcentaje de tráfico está catalogado como anormal? Entiendase por anormal aquel que no está etiquetado como normal.


In [ ]:
# Q4 Mostrar los porcentajes de trafico sobre el total asociados a cada tipo de etiqueta de tráfico 
# (usar el campo genérico Label3 y no el detallado Label 1)
# Mostrar un diagrama con estos porcentajes (bar plot)

In [ ]:
# Q5 Identificar que tipo de tráfico de red está incluido en el dataset (usar el campo ip_proto y convertir 
# ese valor al real que debería tener, es decir, un entero en el rango definido por el IANA)
# https://www.iana.org/assignments/protocol-numbers/protocol-numbers.xhtml


In [ ]:
# Q6 Calcular la cantidad total (suponer que el dato a aculumar para cada paquete es ip_len_mean) 
# de bytes transmitidos agrupados por protocolo

In [ ]:
# Q7 Indicar cual es tráfico anómalo en UDP que usa más tráfico de red. Presentar los resultados en orden demayor a menor


In [ ]:
# Q8 Indicar cual es tráfico anómalo en TCP que usa más tráfico de red. Presentar los resultados en orden demayor a menor